In [ ]:
pip install google-api-python-client pandas openpyxl


In [ ]:
import pandas as pd
import time
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.colab import files

# 1️⃣ Upload file manually
uploaded = files.upload()

master_filename = "kpop_influencers_master.xlsx"
df_master = pd.read_excel(master_filename)
existing_channels = set(df_master["Channel ID"].tolist())
print(f"✅ Loaded {len(existing_channels)} existing influencers to skip duplicates")


Saving kpop_influencers_master (1).xlsx to kpop_influencers_master (1).xlsx
✅ Loaded 5016 existing influencers to skip duplicates


In [ ]:
# 2. Load  file
df_master = pd.read_excel(master_filename)
existing_channels = set(df_master["Channel ID"].tolist())
print(f"✅ Loaded {len(existing_channels)} existing influencers to skip duplicates")

# 3. Setup YouTube API
api_key = "**********************"
youtube = build("youtube", "v3", developerKey=api_key)

# 4. Keywords for fetching new influencers
keywords = [
    "Kpop reaction", "Kpop dance", "Kpop vlog", "Kpop fanclub",
    "Kpop edits", "Kpop live", "Kpop interviews", "Kpop shorts",
    "Kpop newcomers", "Kpop trainee", "Kpop covers", "Kpop challenges"
]

channels = {}  # dictionary for new channels
max_results_per_query = 50
desired_channels = 200  # number of new unique channels to fetch

# 5. Collect unique channels
for query in keywords:
    print(f"🔍 Searching: {query}")
    next_page = None
    while True:
        request = youtube.search().list(
            part="snippet",
            q=query,
            type="channel",
            maxResults=max_results_per_query,
            pageToken=next_page
        )
        response = request.execute()

        for item in response["items"]:
            ch_id = item["id"]["channelId"]
            if ch_id not in channels and ch_id not in existing_channels:
                channels[ch_id] = item["snippet"]["title"]

        next_page = response.get("nextPageToken")
        if not next_page:
            break
    print(f"✅ Found {len(channels)} new so far")
    if len(channels) >= desired_channels:
        break

print(f"🎯 Total new unique channels collected: {len(channels)}")

# 6. Fetch channel stats
data = []
for i, ch_id in enumerate(channels.keys(), 1):
    try:
        request = youtube.channels().list(
            part="snippet,statistics",
            id=ch_id
        )
        response = request.execute()
        for item in response["items"]:
            subscribers = int(item["statistics"].get("subscriberCount", 0))
            views = int(item["statistics"].get("viewCount", 0))
            videos = int(item["statistics"].get("videoCount", 0))

            avg_views_per_video = round(views / videos, 2) if videos > 0 else 0
            subs_per_video = round(subscribers / videos, 2) if videos > 0 else 0
            views_per_subscriber = round(views / subscribers, 2) if subscribers > 0 else 0
            popularity_score = round((subscribers * avg_views_per_video) / 1e6, 2)
            video_saturation = round(videos / subscribers, 6) if subscribers > 0 else 0

            data.append({
                "Channel Name": item["snippet"]["title"],
                "Channel ID": item["id"],
                "Subscribers": subscribers,
                "Views": views,
                "Videos": videos,
                "Avg Views per Video": avg_views_per_video,
                "Subscribers per Video": subs_per_video,
                "Views per Subscriber": views_per_subscriber,
                "Popularity_Score": popularity_score,
                "Video_Saturation": video_saturation
            })
    except Exception as e:
        print("⚠️ Error fetching channel:", e)
    if len(data) % 100 == 0:
        print(f"📈 Processed {len(data)} new channels...")
    if len(data) >= desired_channels:
        break
    time.sleep(0.2)

# 7.  Merge new batch into master
df_new = pd.DataFrame(data)
df_master = pd.concat([df_master, df_new], ignore_index=True)
df_master.drop_duplicates(subset="Channel ID", inplace=True)
print(f"✅ Total unique influencers after merging: {len(df_master)}")

# 8. Save updated master file
updated_filename = "kpop_influencers_master.xlsx"
df_master.to_excel(updated_filename, index=False)

# 9. Download updated file
files.download(updated_filename)

✅ Loaded 4816 existing influencers to skip duplicates
🔍 Searching: Kpop reaction
✅ Found 1 new so far
🔍 Searching: Kpop dance
✅ Found 22 new so far
🔍 Searching: Kpop vlog
✅ Found 92 new so far
🔍 Searching: Kpop fanclub
✅ Found 105 new so far
🔍 Searching: Kpop edits
✅ Found 172 new so far
🔍 Searching: Kpop live
✅ Found 552 new so far
🎯 Total new unique channels collected: 552
📈 Processed 100 new channels...
📈 Processed 200 new channels...
✅ Total unique influencers after merging: 5016


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
duplicates = df_master[df_master.duplicated(subset="Channel ID", keep=False)]
print(f"⚠️ Number of duplicate influencers: {len(duplicates)}")


⚠️ Number of duplicate influencers: 0
